In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
featuretarget= df.drop(['row_id', 'time_id', "investment_id"], axis=1)

In [4]:
corr_matrix=featuretarget.corr()
a=corr_matrix["target"].sort_values()
b=abs(a)
c=b.sort_values()
featurename=list(c.index[50:300])

In [5]:
featurename

['f_292',
 'f_202',
 'f_288',
 'f_60',
 'f_215',
 'f_12',
 'f_217',
 'f_0',
 'f_110',
 'f_253',
 'f_101',
 'f_19',
 'f_282',
 'f_39',
 'f_201',
 'f_283',
 'f_35',
 'f_191',
 'f_176',
 'f_211',
 'f_244',
 'f_242',
 'f_138',
 'f_180',
 'f_149',
 'f_166',
 'f_199',
 'f_279',
 'f_57',
 'f_204',
 'f_50',
 'f_24',
 'f_32',
 'f_18',
 'f_285',
 'f_37',
 'f_218',
 'f_40',
 'f_262',
 'f_6',
 'f_294',
 'f_10',
 'f_261',
 'f_123',
 'f_53',
 'f_161',
 'f_229',
 'f_157',
 'f_222',
 'f_34',
 'f_276',
 'f_236',
 'f_152',
 'f_298',
 'f_22',
 'f_141',
 'f_186',
 'f_140',
 'f_142',
 'f_81',
 'f_127',
 'f_126',
 'f_128',
 'f_95',
 'f_223',
 'f_48',
 'f_97',
 'f_221',
 'f_178',
 'f_213',
 'f_1',
 'f_267',
 'f_130',
 'f_117',
 'f_62',
 'f_86',
 'f_11',
 'f_108',
 'f_208',
 'f_195',
 'f_241',
 'f_196',
 'f_171',
 'f_151',
 'f_159',
 'f_284',
 'f_189',
 'f_131',
 'f_143',
 'f_240',
 'f_156',
 'f_46',
 'f_28',
 'f_14',
 'f_198',
 'f_289',
 'f_89',
 'f_51',
 'f_104',
 'f_59',
 'f_177',
 'f_45',
 'f_38',
 'f_209

In [6]:
np.save('dropout+normrlbatch+250feature/featurename',featurename)

In [8]:
feature = df[featurename]
y = df["target"]
id=df["investment_id"]

In [9]:
import torch
from torch.utils.data import DataLoader,Dataset

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
feature=torch.from_numpy(feature.values)
id=torch.from_numpy(id.values)
y=torch.from_numpy(y.values)
class FinData(Dataset):
    def __init__(self,feature,id,y):
        self.feature = feature.squeeze().to("cuda")
        self.y = y.squeeze().to("cuda")
        self.id = id.squeeze().to(torch.int64).to("cuda")
        self.len = len(y)

    def __getitem__(self,idx):
        return self.id[idx],self.feature[idx],self.y[idx]
  
    def __len__(self):
        return self.len
Dataset_train=FinData(feature=feature,id=id,y=y)
DataLoader1=DataLoader(Dataset_train,batch_size=128,shuffle=True)
DataLoader2=DataLoader(Dataset_train,batch_size=1024,shuffle=True)
DataLoader3=DataLoader(Dataset_train,batch_size=512,shuffle=True)

In [17]:
from torch import nn
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.embedding=nn.Embedding(4000,32)
        self.linear1=nn.Linear(32,64)
        self.linear2=nn.Linear(64,64)
        self.linear3=nn.Linear(250,256)
        self.linear4=nn.Linear(256,256)
        self.linear5=nn.Linear(320,512)
        self.linear6=nn.Linear(512,128)
        self.linear7=nn.Linear(128,32)
        self.linear8=nn.Linear(32,1)
        self.dropout=nn.Dropout(p=0.1)
      

    def forward(self, feature, id):
        id=self.embedding(id).squeeze()
        id=self.linear1(id).squeeze()
        id=self.dropout(id)
        id=id*torch.sigmoid(id)
        id=self.linear2(id).squeeze()
        id=self.dropout(id)
        id=id*torch.sigmoid(id)
        id=self.dropout(id)
        id=self.linear2(id).squeeze()
        id=id*torch.sigmoid(id)


        feature=self.linear3(feature).squeeze()
        feature=self.dropout(feature)
        feature=feature*torch.sigmoid(feature)
        feature=self.linear4(feature).squeeze()
        feature=self.dropout(feature)
        feature=feature*torch.sigmoid(feature)
        feature=self.dropout(feature)
        feature=self.linear4(feature).squeeze()
        feature=feature*torch.sigmoid(feature)

        x=torch.cat((id,feature),1)
        x=self.linear5(x).squeeze()
        x=self.dropout(x)
        x=x*torch.sigmoid(x)
        x=self.linear6(x).squeeze()
        x=self.dropout(x)
        x=x*torch.sigmoid(x)

        x=self.linear7(x).squeeze()
        x=self.dropout(x)
        x=x*torch.sigmoid(x)

        x=self.linear8(x).squeeze()
        x=40*torch.sigmoid(x)-20
        return x.squeeze()

In [18]:
model1_1 = NeuralNetwork().to("cuda")
model1_1=model1_1.double()
model1_2 = NeuralNetwork().to("cuda")
model1_2=model1_2.double()

model2_1 = NeuralNetwork().to("cuda")
model2_1=model2_1.double()
model2_2 = NeuralNetwork().to("cuda")
model2_2=model2_2.double()


model3_1 = NeuralNetwork().to("cuda")
model3_1=model3_1.double()
model3_2 = NeuralNetwork().to("cuda")
model3_2=model3_2.double()

In [19]:
learning_rate = 1e-3
optimizer1_1 = torch.optim.Adam(model1_1.parameters(), lr=learning_rate,weight_decay=1e-4)
optimizer1_2 = torch.optim.Adam(model1_2.parameters(), lr=learning_rate,weight_decay=1e-3)
optimizer2_1 = torch.optim.Adam(model2_1.parameters(), lr=learning_rate,weight_decay=1e-4)
optimizer2_2 = torch.optim.Adam(model2_2.parameters(), lr=learning_rate,weight_decay=1e-3)
optimizer3_1 = torch.optim.Adam(model3_1.parameters(), lr=learning_rate,weight_decay=1e-4)
optimizer3_2 = torch.optim.Adam(model3_2.parameters(), lr=learning_rate,weight_decay=1e-3)
loss_fn1=torch.nn.MSELoss()

In [20]:
from tqdm import tqdm

In [21]:
def train_1(dataloader, model, optimizer):
    for  id,feature,y in tqdm(dataloader):
        # Compute prediction and loss
        pred = model(feature,id)
        loss = loss_fn1(pred,y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
def train_2(dataloader, model, optimizer):
    for id,feature, y in tqdm(dataloader):
        # Compute prediction and loss
        pred = model(feature,id)
        loss = -torch.corrcoef(torch.stack((pred,y)))[0,1]
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [22]:
for i in range(5):
    print("epoch{}/{}".format(i+1,5))
    train_1(DataLoader1,model1_1,optimizer1_1)
    train_2(DataLoader1,model1_2,optimizer1_2)
    train_1(DataLoader2,model2_1,optimizer2_1)
    train_2(DataLoader2,model2_2,optimizer2_2)
    train_1(DataLoader3,model3_1,optimizer3_1)
    train_2(DataLoader3,model3_2,optimizer3_2)

epoch1/5


100%|██████████| 6136/6136 [01:12<00:00, 85.03it/s]


epoch2/5


100%|██████████| 6136/6136 [01:12<00:00, 84.67it/s]


epoch3/5


100%|██████████| 6136/6136 [01:12<00:00, 84.83it/s]


epoch4/5


100%|██████████| 6136/6136 [01:12<00:00, 84.54it/s]


epoch5/5


100%|██████████| 6136/6136 [01:12<00:00, 84.40it/s]


In [23]:
torch.save(model1_1, 'dropout+normrlbatch+250feature/model1_1.pth')
torch.save(model1_2, 'dropout+normrlbatch+250feature/model1_2.pth')
torch.save(model2_1, 'dropout+normrlbatch+250feature/model2_1.pth')
torch.save(model2_2, 'dropout+normrlbatch+250feature/model2_2.pth')
torch.save(model3_1, 'dropout+normrlbatch+250feature/model3_1.pth')
torch.save(model3_2, 'dropout+normrlbatch+250feature/model3_2.pth')